# Milestone 2 - Analysis Pipeline

## Adding all necessary imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import time

## Importing Data sets

In [3]:
game_shifts = pd.read_csv("/Users/vanessadunn/Documents/School/4thYear/Data301/game_shifts.csv")
game_shifts.head()

,game_id,player_id,period,shift_start,shift_end
0,2018020001,8466139,1,0,42.0
1,2018020001,8466139,1,207,247.0
2,2018020001,8466139,1,375,413.0
3,2018020001,8466139,1,556,574.0
4,2018020001,8466139,1,605,631.0


In [4]:
game_stats = pd.read_csv("/Users/vanessadunn/Documents/School/4thYear/Data301/game_skater_stats.csv")
game_stats.head()

,game_id,player_id,team_id,timeOnIce,assists,goals,shots,hits,powerPlayGoals,powerPlayAssists,...,faceoffTaken,takeaways,giveaways,shortHandedGoals,shortHandedAssists,blocked,plusMinus,evenTimeOnIce,shortHandedTimeOnIce,powerPlayTimeOnIce
0,2016020045,8468513,4,955,1,0,0,2.0,0,0,...,0,1.0,1.0,0,0,1.0,1,858,97,0
1,2016020045,8476906,4,1396,1,0,4,2.0,0,0,...,0,1.0,2.0,0,0,2.0,0,1177,0,219
2,2016020045,8474668,4,915,0,0,1,1.0,0,0,...,0,2.0,0.0,0,0,0.0,-1,805,0,110
3,2016020045,8473512,4,1367,3,0,0,0.0,0,2,...,27,0.0,0.0,0,0,0.0,-1,1083,19,265
4,2016020045,8471762,4,676,0,0,3,2.0,0,0,...,0,0.0,1.0,0,0,0.0,-1,613,63,0


Removing unecessary columns

In [5]:
gamestat = game_stats.drop(['faceoffTaken', 'takeaways', 'giveaways','hits','blocked','faceOffWins','evenTimeOnIce','penaltyMinutes','powerPlayTimeOnIce','shortHandedTimeOnIce'],axis='columns')
gamestat["Goals"]=(gamestat["goals"]+gamestat["powerPlayGoals"]+gamestat["shortHandedGoals"])
gamestat["Assists"]=(gamestat["assists"]+gamestat["powerPlayAssists"]+gamestat["shortHandedAssists"])


In [16]:
###Calculting the length of shifts for each of the players in each game
game_shifts['shiftlength']=(game_shifts['shift_end']-game_shifts['shift_start'])

In [7]:
game_shifts['game_id']=game_shifts['game_id'].values.astype(str)
game_shifts['player_id']=game_shifts['player_id'].values.astype(str)

In [8]:
#Creating a common ID for the players/teams
game_shifts['ID']=game_shifts['game_id'].str.cat(game_shifts['player_id'],sep='-')
game_shifts.head()

,game_id,player_id,period,shift_start,shift_end,shiftlength,ID
0,2018020001,8466139,1,0,42.0,42.0,2018020001-8466139
1,2018020001,8466139,1,207,247.0,40.0,2018020001-8466139
2,2018020001,8466139,1,375,413.0,38.0,2018020001-8466139
3,2018020001,8466139,1,556,574.0,18.0,2018020001-8466139
4,2018020001,8466139,1,605,631.0,26.0,2018020001-8466139


In [9]:
#Taking average for each players shift length in the given games
shiftscombined = game_shifts.groupby('ID').mean()
shiftscleaned=shiftscombined.drop(['period','shift_start','shift_end'], axis='columns')
shiftsclean = shiftscleaned[shiftscleaned['shiftlength']>0]
shiftsclean

,shiftlength
ID,
2010020001-8459628,45.966667
2010020001-8462044,504.142857
2010020001-8465200,55.192308
2010020001-8467351,48.739130
2010020001-8467400,46.241379
...,...
2019030416-8479381,37.071429
2019030416-8479410,35.666667
2019030416-8480036,51.129032


In [10]:
gamestat['game_id']=gamestat['game_id'].values.astype(str)
gamestat['player_id']=gamestat['player_id'].values.astype(str)
gamestat['ID']=gamestat['game_id'].str.cat(gamestat['player_id'],sep='-')

In [11]:
statscleaned=gamestat.drop(['goals','assists','powerPlayGoals','powerPlayAssists','shortHandedGoals','shortHandedAssists','game_id','player_id'], axis='columns')

In [12]:
#Using Join Inner
df=pd.merge(statscleaned,shiftsclean,how='inner',on ='ID')
df=df[['ID','shiftlength','team_id','shots','Goals','Assists','plusMinus','timeOnIce']]
df

,ID,shiftlength,team_id,shots,Goals,Assists,plusMinus,timeOnIce
0,2016020045-8468513,35.370370,4,0,0,1,1,955
1,2016020045-8476906,49.857143,4,4,0,1,0,1396
2,2016020045-8474668,43.571429,4,1,0,0,-1,915
3,2016020045-8473512,47.137931,4,0,0,5,-1,1367
4,2016020045-8471762,37.555556,4,3,0,0,-1,676
...,...,...,...,...,...,...,...,...
544152,2018030417-8475745,41.500000,6,2,0,0,0,913
544153,2018030417-8478443,38.692308,6,1,0,0,-1,1006
544154,2018030417-8478443,38.692308,6,1,0,0,-1,1006
544155,2018030417-8478046,41.318182,6,2,0,0,0,909


In [13]:
df_goals=df[df['Goals']>0]
df_goals

,ID,shiftlength,team_id,shots,Goals,Assists,plusMinus,timeOnIce
12,2016020045-8476461,40.444444,4,2,1,0,-1,1092
14,2016020045-8476177,39.730769,4,3,3,0,-1,1033
17,2016020045-8474190,45.724138,4,6,2,0,-1,1326
18,2016020045-8466148,42.529412,16,2,2,0,0,723
20,2016020045-8477929,36.222222,16,1,1,0,1,652
...,...,...,...,...,...,...,...,...
544112,2018030417-8475158,42.074074,19,2,1,1,2,1136
544119,2018030417-8475170,37.807692,19,3,1,0,2,983
544120,2018030417-8475170,37.807692,19,3,1,0,2,983
544137,2018030417-8476891,39.851852,6,2,1,0,-1,1076


# EDA

In [14]:
print(f"the min is {min(df['shiftlength'])}, the max is {max(df['shiftlength'])}")

the min is 1.0, the max is 105.84210526315789


In [15]:
df.describe()

,shiftlength,team_id,shots,Goals,Assists,plusMinus,timeOnIce
count,544157.000000,544157.000000,544157.000000,544157.00000,544157.000000,544157.000000,544157.000000
mean,45.787846,17.813094,1.702873,0.19288,0.330234,-0.007992,993.572298
std,6.505445,12.488773,1.544268,0.53692,0.713281,1.118561,294.654707
min,1.000000,1.000000,0.000000,0.00000,0.000000,-7.000000,1.000000
25%,41.391304,8.000000,1.000000,0.00000,0.000000,-1.000000,798.000000
50%,45.473684,17.000000,1.000000,0.00000,0.000000,0.000000,998.000000
75%,49.843750,24.000000,3.000000,0.00000,0.000000,1.000000,1191.000000
max,105.842105,54.000000,15.000000,7.00000,9.000000,6.000000,3906.000000
